# Notebook for transformers exploration

In [ ]:
# flake8-noqa-cell
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moralization import input as inp
from moralization import analyse as ae

In [ ]:
!nvidia-smi

In [ ]:
# flake8-noqa-cell
raw_data_no_moralization = (
    "../../moralization-notes/Alle_bearbeiteten_Annotationen-0_label.csv"
)
df_raw_no_moralization = pd.read_csv(raw_data_no_moralization)
data_dict = inp.InputOutput.read_data("../data/All_Data/XMI_11")
df_spans = ae.AnalyseOccurrence(data_dict, mode="spans").df

In [ ]:
df_spans.head(10)

In [ ]:
df_raw_no_moralization = df_raw_no_moralization.rename(
    columns={"Label": "Label_moralization"}
)

In [ ]:
df_raw_no_moralization.head(10)

In [ ]:
df_spans.loc["KAT1-Moralisierendes Segment"]
# all that are not "Keine Moralisierung" shall be "Moralisierung"

In [ ]:
# flake8-noqa-cell
df_new = df_spans.loc[["KAT1-Moralisierendes Segment"]]
# drop the multiindex
df_new = df_new.droplevel(0)
# sum strings over all sources
df_new["All sources"] = ""
for file in df_new.columns[:-1]:
    print(file)
    df_new["All sources"] += df_new[file] + "###"
# extract row content into new dataframe
# no moralization
df_no_moralization = pd.DataFrame(
    df_new["All sources"].loc["Keine Moralisierung"].split("###"), columns=["Sentences"]
)
# drop empty rows
df_no_moralization = df_no_moralization[df_no_moralization["Sentences"].astype(bool)]
df_no_moralization["Label_moralization"] = 0
# moralization
df_moralization = pd.DataFrame(
    df_new["All sources"].loc["Moralisierung"].split("###"), columns=["Sentences"]
)
df_moralization = df_moralization[df_moralization["Sentences"].astype(bool)]
df_moralization["Label_moralization"] = 1
df_moralization_exp = pd.DataFrame(
    df_new["All sources"].loc["Moralisierung explizit"].split("###"),
    columns=["Sentences"],
)
df_moralization_exp = df_moralization_exp[df_moralization_exp["Sentences"].astype(bool)]
df_moralization_exp["Label_moralization"] = 1
df_moralization_int = pd.DataFrame(
    df_new["All sources"].loc["Moralisierung interpretativ"].split("###"),
    columns=["Sentences"],
)
df_moralization_int = df_moralization_int[df_moralization_int["Sentences"].astype(bool)]
df_moralization_int["Label_moralization"] = 1
df_new.to_csv("df_new.csv")

In [ ]:
df_new.head()

In [ ]:
df_raw_no_moralization.head(10)

In [ ]:
# merge all the data frames into one
frames = [
    df_raw_no_moralization,
    df_no_moralization,
    df_moralization,
    df_moralization_exp,
    df_moralization_int,
]
all_data = pd.concat(frames)

In [ ]:
all_data.head(100)

### Inspect the data

In [ ]:
all_data["Label_no_moralization"] = np.where(all_data["Label_moralization"] == 1, 0, 1)
all_data[["Label_moralization", "Label_no_moralization"]].sum().plot.bar()

In [ ]:
all_data[["Label_moralization", "Label_no_moralization"]].sum().plot.bar(
    ylim=([0, 2000])
)

## Now we got the data in one frame, let's reshuffle and split into train, test, validate